# Recommendation Systems

In [3]:
import pandas as pd

# Let's take a sample here
SAMPLE_SIZE = 2000

# Import Movielens 100k Database
# https://grouplens.org/datasets/movielens/
# Don't forget to change the file path
raw_movies = pd.read_csv("data/ml-25m/movies.csv", )
# If you wish to use the full dataset, just remove the nrows argument
raw_ratings = pd.read_csv("data/ml-25m/ratings.csv", nrows=SAMPLE_SIZE)

In [ ]:

from jai import Trainer
from jai.utilities import split


# Fix Movies Data
bases, movies = split(raw_movies, {"genres": "|", "title": None})
movies.index = movies["movieId"]

ratings = raw_ratings.drop("timestamp", axis=1)
users = pd.DataFrame(
    {"userId": ratings["userId"].unique().astype(str)}, index=ratings["userId"].unique()
)


In [ ]:

### Send Movie Titles, Genre and Tag Data to JAI
trainer = Trainer(name="movie_titles")
trainer.set_parameters(db_type="Text")
q_title = trainer.fit(bases["title"], overwrite=True)


In [ ]:


# movies genre
trainer = Trainer(name="genre")
trainer.set_parameters(db_type="Text")
q_genre = trainer.fit(bases["genres"], overwrite=True)


In [ ]:

#%%
trainer = Trainer(name="ratings")
trainer.set_parameters(
    db_type="RecommendationSystem",
    hyperparams={"batch_size": 0, "model": {"hidden_latent_dim": 512}},
    features={"userId": {"name": "userId", "dtype": "category"}},
    pretrained_bases=[
        {"id_name": "movieId", "db_parent": "movies"},
        {"id_name": "userId", "db_parent": "users"},
        {"id_name": "id_title", "db_parent": "movie_titles"},
        {"id_name": "id_genres", "db_parent": "genre"},
    ],
    label={"label_name": "rating"},
)


In [ ]:
trainer.setup_parameters

In [ ]:

queries = trainer.fit(
    {"users": users, "movies": movies, "main": ratings},
    overwrite=True,
)


In [ ]:
# r = j.recommendation("movies", [uid], top_k=100)


In [ ]:

# dfr = pd.DataFrame(r[0]['results'])
# dfr = dfr.merge(raw_movies, how="left",
#                   left_on="id",
#                   right_on="movieId")
# dfr

# s = j.similar("users", [uid], top_k=100)
# dfs = pd.DataFrame(s[0]['results'])

# %%
